In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

In [2]:
# Reading input

articles = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
# customers = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')
transactions_train = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
sample_submission = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')\
[['customer_id']]

In [3]:
transactions_train['customer_id'].nunique(), sample_submission.shape[0]

(1362281, 1371980)

In [4]:
pd.merge(transactions_train['customer_id'].drop_duplicates(),
         sample_submission['customer_id'],
         left_on = 'customer_id',
         right_on = 'customer_id'
        ).shape

(1362281, 1)

In [5]:
print(articles['article_id'].nunique(),
articles['prod_name'].nunique(),
articles['product_type_name'].nunique(),
articles['product_group_name'].nunique() )

105542 45875 131 19


In [6]:
print(articles['graphical_appearance_name'].nunique(),
articles['colour_group_name'].nunique(),
articles['department_name'].nunique() )

30 50 250


In [7]:
print(articles['index_name'].nunique(),
articles['index_group_name'].nunique(),
articles['section_name'].nunique(),
articles['garment_group_name'].nunique(),
articles['detail_desc'].nunique()
     )

10 5 56 21 43404


In [8]:
articles.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [9]:
articles[['product_type_name']].drop_duplicates()

,product_type_name
0,Vest top
3,Bra
6,Underwear Tights
7,Socks
8,Leggings/Tights
...,...
102766,Dog wear
103628,Eyeglasses
103910,Wireless earphone case
104253,Stain remover spray


In [10]:
transactions_train['t_dat_datetime']  = pd.to_datetime(transactions_train['t_dat'])

transactions_train['t_month'] = transactions_train['t_dat_datetime'].dt.month

transactions_train['t_year']  = transactions_train['t_dat_datetime'].dt.year

In [11]:
transactions_train['max_purc_date'] = transactions_train.groupby('customer_id')['t_dat_datetime'].transform('max')

transactions_train['min_purc_date'] = transactions_train.groupby('customer_id')['t_dat_datetime'].transform('min')

transactions_train['total_visits'] = transactions_train.groupby('customer_id')['t_dat'].transform('nunique')

transactions_train['total_item_bought'] = transactions_train.groupby('customer_id')['article_id'].transform('nunique')

transactions_train['max_year'] = transactions_train.groupby('customer_id')['t_year'].transform('max')

In [12]:
# transactions_train_filt = transactions_train[transactions_train['max_year'] == 2020]

# transactions_train_filt['product_reach'] = transactions_train_filt.groupby('article_id')['customer_id'].transform('nunique')

# transactions_train_filt = transactions_train_filt[transactions_train_filt['product_reach'] > 8]

transactions_train['repeat_purchase'] = transactions_train.groupby(['customer_id','article_id'])['t_dat'].transform('nunique')

transactions_train_filt = transactions_train[transactions_train['repeat_purchase'] > 1]

# transactions_train_filt = transactions_train_filt[transactions_train_filt['total_visits'] >= 2]

# transactions_train_filt = transactions_train_filt[transactions_train_filt['total_item_bought'] >= 12]

In [13]:
article_cust = transactions_train_filt[['article_id','customer_id']].drop_duplicates()

In [14]:
article_cust_full = pd.merge(article_cust, article_cust.rename(columns={'article_id':'article_id_1',
                                                   }),
         left_on='customer_id',
         right_on='customer_id'
        )

In [15]:
article_cust_full = article_cust_full[article_cust_full['article_id'] != article_cust_full['article_id_1']]

In [16]:
common_cust = article_cust_full.groupby(['article_id', 'article_id_1']).\
agg(common_cust=('customer_id','nunique')).reset_index()

In [17]:
common_cust.shape

(8510616, 3)

In [18]:
common_cust = pd.merge(common_cust,
                       article_cust_full.groupby('article_id').\
                       agg(article_id_cus=('customer_id','nunique')).\
                       reset_index(),
                       left_on = 'article_id',
                       right_on = 'article_id'
                      )
common_cust.head()

,article_id,article_id_1,common_cust,article_id_cus
0,108775015,108775044,50,338
1,108775015,111565001,2,338
2,108775015,111586001,3,338
3,108775015,111593001,5,338
4,108775015,120129001,2,338


In [19]:
common_cust.shape

(8510616, 4)

In [20]:
common_cust = pd.merge(common_cust,
                       article_cust_full.groupby('article_id_1').\
                       agg(article_id_cus=('customer_id','nunique')).\
                       reset_index(),
                       left_on = 'article_id_1',
                       right_on = 'article_id_1'
                      )
common_cust.head()

,article_id,article_id_1,common_cust,article_id_cus_x,article_id_cus_y
0,108775015,108775044,50,338,203
1,110065001,108775044,1,24,203
2,110065002,108775044,1,13,203
3,111565001,108775044,1,153,203
4,123173001,108775044,1,391,203


In [21]:
common_cust['lift'] = common_cust['common_cust']/ (common_cust['article_id_cus_x']*common_cust['article_id_cus_y'])

In [22]:
common_cust = common_cust[common_cust['common_cust']>10].sort_values(['article_id_cus_y','lift'], ascending=[False,False])

In [23]:
len(common_cust['article_id'].unique())

2155

In [24]:
common_cust = pd.merge(common_cust,
         transactions_train_filt.groupby(['article_id']).\
         agg(article_id_max_dat=('t_dat_datetime','max')).\
         reset_index(),
         left_on='article_id',
         right_on='article_id'
        )

In [25]:
common_cust = pd.merge(common_cust,
         transactions_train_filt.rename(columns={}).groupby(['article_id']).\
         agg(article_id_1_max_dat=('t_dat_datetime','max')).\
         reset_index(),
         left_on='article_id_1',
         right_on='article_id'
        )

In [26]:
common_cust.drop(['article_id_y'], axis = 1, inplace=True)
common_cust.head()

,article_id_x,article_id_1,common_cust,article_id_cus_x,article_id_cus_y,lift,article_id_max_dat,article_id_1_max_dat
0,673901001,706016001,82,212,5023,0.000077,2020-02-06,2020-09-22
1,706016019,706016001,106,280,5023,0.000075,2020-09-22,2020-09-22
2,706016028,706016001,17,46,5023,0.000074,2020-09-22,2020-09-22
3,706016006,706016001,312,853,5023,0.000073,2020-09-22,2020-09-22
4,539723001,706016001,153,431,5023,0.000071,2019-09-18,2020-09-22


In [27]:
len(transactions_train_filt[(transactions_train_filt['t_month'].isin([8,9,10]))&
                            (transactions_train_filt['t_year']!=2020)
                            
                           ]['customer_id'].unique())

140303

In [29]:
product_purchase_cycle = pd.merge(transactions_train_filt[['customer_id','article_id','t_dat_datetime']],
                                  articles[['article_id','product_type_no']],
                                  left_on='article_id',
                                  right_on='article_id'
                                 ).\
sort_values(['customer_id','article_id','t_dat_datetime']).drop_duplicates()

In [30]:
product_purchase_cycle['t_dat_datetime_lag'] = product_purchase_cycle.\
groupby(['customer_id','article_id'])['t_dat_datetime'].shift(1)

In [31]:
product_purchase_cycle['purchase_cycle'] = product_purchase_cycle['t_dat_datetime'] - product_purchase_cycle['t_dat_datetime_lag']

In [32]:
Average_Purchase_Cycle = product_purchase_cycle[~(product_purchase_cycle['purchase_cycle'].isnull())].\
groupby(['customer_id','article_id']).agg({"purchase_cycle":"median"}).reset_index()

In [33]:
transactions_train_filt = pd.merge(pd.merge(transactions_train_filt,
                                  articles[['article_id','product_type_no']],
                                  left_on='article_id',
                                  right_on='article_id'
                                 ),
                                   Average_Purchase_Cycle,
                                   left_on = ['customer_id','article_id'],
                                   right_on = ['customer_id','article_id'],
                                   how='inner'
                                  )

In [34]:
transactions_train_filt['product_last_purchase_date'] = transactions_train_filt.\
groupby(['customer_id','article_id'])['t_dat_datetime'].transform("max")

In [35]:
from datetime import datetime
from datetime import timedelta

In [36]:
transactions_train_filt['purchase_cycle_days'] = transactions_train_filt['purchase_cycle'].astype('str').str.split(' ', expand=True)[0]

In [37]:
transactions_train_filt['next_purchase_date'] = transactions_train_filt['product_last_purchase_date'] + transactions_train_filt['purchase_cycle_days'].astype(int).map(timedelta)

In [38]:
start_date = transactions_train_filt['t_dat_datetime'].max() + timedelta(-90)
end_date = transactions_train_filt['t_dat_datetime'].max() + timedelta(90)

In [39]:
datetime.date(transactions_train_filt['t_dat_datetime'].max()) 

datetime.date(2020, 9, 22)

In [40]:
start_date, end_date

(Timestamp('2020-06-24 00:00:00'), Timestamp('2020-12-21 00:00:00'))

In [41]:
len(transactions_train_filt['customer_id'].unique())

377478

In [61]:
customer_next_purchase = transactions_train_filt[transactions_train_filt['next_purchase_date'].\
                        between(start_date, end_date, inclusive="both")]\
[['customer_id','t_dat','article_id']].drop_duplicates()

In [62]:
customer_next_purchase['repeat_purchase'] = customer_next_purchase.groupby(['customer_id'])['article_id'].transform('count')

In [63]:
customer_next_purchase['product_imp_rank'] = customer_next_purchase.groupby(['customer_id'])['repeat_purchase'].rank(method="first",ascending=True)

In [64]:
customer_next_purchase['max_rank'] = customer_next_purchase[customer_next_purchase['product_imp_rank'] <= 12].\
groupby('customer_id')['product_imp_rank'].transform('max')

In [65]:
customer_next_purchase = customer_next_purchase[['customer_id','article_id','max_rank']]

customer_next_purchase = customer_next_purchase.sort_values(['customer_id','article_id'])

In [66]:
len(customer_next_purchase['customer_id'].unique())

117328

In [48]:
common_cust['left_rank'] = common_cust[common_cust.article_id_1_max_dat.dt.year==2020].\
groupby(['article_id_x'])['lift'].rank(method='first', ascending=False)

In [50]:
# common_cust.sort_values(['article_id_x','lift'], ascending=[False,False])

In [52]:
customer_next_purchase.shape

(437842, 3)

In [67]:
customer_next_purchase = pd.merge(customer_next_purchase, 
         common_cust[common_cust['left_rank']==1][['article_id_x','article_id_1']].\
rename(columns={'article_id_x':'article_id','article_id_1':'sub_prod'}),
         left_on = 'article_id',
         right_on = 'article_id',
         how = 'left'
        )

In [69]:
customer_next_purchase['article_id'] = np.where(customer_next_purchase['sub_prod'].isnull(),
                                                customer_next_purchase['article_id'],
                                                customer_next_purchase['sub_prod']
                                               ).astype('int')

In [72]:
# Filling Products 

top_12_products = customer_next_purchase.groupby(['article_id']).agg(cust_count=('customer_id','nunique')).reset_index()

top_12_products = top_12_products.sort_values('cust_count', ascending = False).head(12)[['article_id']]

In [73]:
filler_cust_prod_comb= pd.merge(customer_next_purchase['customer_id'].drop_duplicates(), 
top_12_products['article_id'], how = 'cross')

In [74]:
customer_next_purchase= pd.concat([customer_next_purchase[['customer_id', 'article_id']],
          filler_cust_prod_comb[['customer_id', 'article_id']]],
          axis = 0
         ).drop_duplicates()

In [75]:
customer_next_purchase['row_num'] = customer_next_purchase.groupby(['customer_id']).cumcount()

In [76]:
customer_next_purchase = customer_next_purchase[customer_next_purchase['row_num'] <= 11]

In [77]:
customer_next_purchase['article_id'] = ' 0'+customer_next_purchase['article_id'].astype('str')

In [78]:
customer_next_purchase = customer_next_purchase.groupby(['customer_id'])['article_id'].sum().reset_index()

In [79]:
top_12_products['article_id'] = ' 0'+top_12_products['article_id'].astype('str')

In [80]:
top_12_products_for_missin_cust = top_12_products['article_id'].sum()
top_12_products_for_missin_cust

' 0673901001 0706016019 0706016006 0706016001 0372860001 0372860002 0610776083 0160442007 0464297031 0160442010 0751471042 0399223024'

In [81]:
sample_submission = pd.merge(sample_submission,
                             customer_next_purchase,
                             left_on = ['customer_id'],
                             right_on = ['customer_id'],
                             how = 'left'
                            ).fillna(top_12_products_for_missin_cust)

In [ ]:
# no_products_cust['article_id'].fillna( "0706016001 0706016002 0372860001 0399223001 0464297007 0160442007 0610776002 0156231001 0399256001 0372860002 0706016003 0759871002", inplace=True)
# no_products_cust.shape

In [83]:
sample_submission.rename(columns={'article_id':'prediction'}, inplace=True)
sample_submission.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0673901001 0706016019 0706016006 0706016001 0...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0723529001 0351484002 0673901001 0706016019 0...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0673901001 0706016019 0706016006 0706016001 0...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0673901001 0706016019 0706016006 0706016001 0...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0673901001 0706016019 0706016006 0706016001 0...


In [84]:
sample_submission.to_csv('submission.csv', index =False)